In [1]:
# Importing necessary libraries
import pandas as pd

In [3]:
# Load the cleaned dataset (adjust path as needed)
df = pd.read_csv('cleaned_digital_wallet_transactions.csv')

In [5]:
# Fraud Indicator 1: High Transaction Amounts with Statistical Threshold
threshold_amount = 5000  # Set an initial threshold for high transaction amounts
# Calculate mean and standard deviation of transaction amounts for enhanced analysis
mean_amount = df['product_amount'].mean()
std_amount = df['product_amount'].std()
upper_limit = mean_amount + 3 * std_amount  # Setting a dynamic threshold (3-sigma rule)

high_transaction_flag = df['product_amount'] > upper_limit

if high_transaction_flag.any():
    print("High Transaction Amounts Detected:")
    for _, row in df[high_transaction_flag].iterrows():
        print("User ID:", row['user_id'],
              "| Transaction Amount:", f"${row['product_amount']:.2f}",
              "| Date:", row['transaction_date'],
              "| Transaction ID:", row['transaction_id'],
              "| Reason: Exceeds 3-sigma threshold (Mean:", f"${mean_amount:.2f},", "Std Dev:", f"${std_amount:.2f})")


In [9]:
# Fraud Indicator 2: Frequent Transactions in Short Time Periods (Behavioral Analysis)
df['transaction_date'] = pd.to_datetime(df['transaction_date'])  # Ensure datetime format
df = df.sort_values(by=['user_id', 'transaction_date'])  # Sort by user and time
df['time_diff'] = df.groupby('user_id')['transaction_date'].diff().dt.total_seconds() / 60  # in minutes
frequent_transaction_flag = (df['time_diff'] < 10)  # Transactions within 10 minutes

if frequent_transaction_flag.any():
    print("Frequent Transactions Detected:")
    for _, row in df[frequent_transaction_flag].iterrows():
        print("User ID:", row['user_id'],
              "| Consecutive Transactions within", f"{int(row['time_diff'])} minutes",
              "| Date:", row['transaction_date'],
              "| Transaction ID:", row['transaction_id'],
              "| Reason: Rapid transactions may suggest automated activity.")

In [11]:
# Fraud Indicator 3: Unusual Location for User's Transactions (Location Matching and Profiling)
df['most_common_location'] = df.groupby('user_id')['location'].transform(lambda x: x.mode()[0])
unusual_location_flag = df['location'] != df['most_common_location']

if unusual_location_flag.any():
    print("Unusual Locations Detected:")
    for _, row in df[unusual_location_flag].iterrows():
        print("User ID:", row['user_id'],
              "| Transaction Location:", row['location'],
              "| Usual Location:", row['most_common_location'],
              "| Date:", row['transaction_date'],
              "| Transaction ID:", row['transaction_id'],
              "| Reason: Location mismatch may indicate suspicious activity.")

Unusual Locations Detected:
User ID: USER_00032 | Transaction Location: urban | Usual Location: suburban | Date: 2023-11-30 23:01:00 | Transaction ID: b7d27ac5-043e-4996-8d29-ee2a7de1d885 | Reason: Location mismatch may indicate suspicious activity.
User ID: USER_00061 | Transaction Location: urban | Usual Location: suburban | Date: 2023-12-17 15:05:00 | Transaction ID: 5c217dbd-4982-425f-b547-e2071764ca85 | Reason: Location mismatch may indicate suspicious activity.
User ID: USER_00069 | Transaction Location: urban | Usual Location: suburban | Date: 2024-06-24 17:38:00 | Transaction ID: ad96139d-1bad-40be-a114-c64e3e66e37c | Reason: Location mismatch may indicate suspicious activity.
User ID: USER_00099 | Transaction Location: urban | Usual Location: suburban | Date: 2023-12-19 21:49:00 | Transaction ID: d032dbfb-06bd-4412-b170-eefaa8258c40 | Reason: Location mismatch may indicate suspicious activity.
User ID: USER_00107 | Transaction Location: urban | Usual Location: rural | Date: 20

In [13]:
# Fraud Indicator 4: Irregular Product Category Purchases (Behavioral Profiling and Clustering)
df['usual_product_category'] = df.groupby('user_id')['product_category'].transform(lambda x: x.mode()[0])
irregular_product_flag = df['product_category'] != df['usual_product_category']

if irregular_product_flag.any():
    print("Irregular Product Categories Detected:")
    for _, row in df[irregular_product_flag].iterrows():
        print("User ID:", row['user_id'],
              "| Product Category:", row['product_category'],
              "| Usual Category:", row['usual_product_category'],
              "| Date:", row['transaction_date'],
              "| Transaction ID:", row['transaction_id'],
              "| Reason: Purchase outside usual category may signal unusual account activity.")

Irregular Product Categories Detected:
User ID: USER_00019 | Product Category: mobile recharge | Usual Category: food delivery | Date: 2023-10-08 02:13:00 | Transaction ID: be4c05fe-b8b8-4c42-84e4-8f02c8f21cd7 | Reason: Purchase outside usual category may signal unusual account activity.
User ID: USER_00028 | Product Category: mobile recharge | Usual Category: insurance premium | Date: 2024-06-01 18:56:00 | Transaction ID: 527720b9-6104-4646-84fd-b8ca705bcf7e | Reason: Purchase outside usual category may signal unusual account activity.
User ID: USER_00028 | Product Category: streaming service | Usual Category: insurance premium | Date: 2024-06-12 07:26:00 | Transaction ID: 04ce452d-d5c4-457c-88ea-35c944de8435 | Reason: Purchase outside usual category may signal unusual account activity.
User ID: USER_00028 | Product Category: online shopping | Usual Category: insurance premium | Date: 2024-07-22 05:19:00 | Transaction ID: 6151f531-2e39-4e4b-9f02-67daa4e3c5fb | Reason: Purchase outside

In [15]:
# Fraud Indicator 5: Multiple Payment Methods for a Single User (Behavioral Consistency Check)
payment_method_count = df.groupby('user_id')['payment_method'].nunique()
multiple_payment_method_users = payment_method_count[payment_method_count > 1].index
multiple_payment_method_flag = df['user_id'].isin(multiple_payment_method_users)

if multiple_payment_method_flag.any():
    print("Multiple Payment Methods Detected:")
    for _, row in df[multiple_payment_method_flag].iterrows():
        print("User ID:", row['user_id'],
              "| Payment Method:", row['payment_method'],
              "| Date:", row['transaction_date'],
              "| Transaction ID:", row['transaction_id'],
              "| Reason: Multiple payment methods may indicate account takeover or fraud.")

Multiple Payment Methods Detected:
User ID: USER_00019 | Payment Method: debit card | Date: 2023-10-08 02:13:00 | Transaction ID: be4c05fe-b8b8-4c42-84e4-8f02c8f21cd7 | Reason: Multiple payment methods may indicate account takeover or fraud.
User ID: USER_00019 | Payment Method: bank transfer | Date: 2024-03-17 10:13:00 | Transaction ID: 11f6179c-9e7c-4104-b6f7-e936a336f352 | Reason: Multiple payment methods may indicate account takeover or fraud.
User ID: USER_00028 | Payment Method: bank transfer | Date: 2023-08-28 13:13:00 | Transaction ID: 78b81362-4180-43e2-a794-97888a41faa5 | Reason: Multiple payment methods may indicate account takeover or fraud.
User ID: USER_00028 | Payment Method: upi | Date: 2024-06-01 18:56:00 | Transaction ID: 527720b9-6104-4646-84fd-b8ca705bcf7e | Reason: Multiple payment methods may indicate account takeover or fraud.
User ID: USER_00028 | Payment Method: wallet balance | Date: 2024-06-12 07:26:00 | Transaction ID: 04ce452d-d5c4-457c-88ea-35c944de8435 | 

In [17]:
# Enhanced Summary Report with Statistical Insight
summary = {
    "High Transaction Amounts": high_transaction_flag.sum(),
    "Frequent Transactions in Short Time Periods": frequent_transaction_flag.sum(),
    "Unusual Locations for User's Transactions": unusual_location_flag.sum(),
    "Irregular Product Category Purchases": irregular_product_flag.sum(),
    "Multiple Payment Methods for a Single User": multiple_payment_method_flag.sum()
}

In [19]:
# In-depth Decision Summary
print("\n--- Decision Summary ---")
if summary["High Transaction Amounts"] > 0:
    print(str(summary["High Transaction Amounts"]) + " transactions exceeded the 3-sigma threshold for transaction amount."
          " Review large transactions for high-risk patterns that align with known fraud techniques.")
if summary["Frequent Transactions in Short Time Periods"] > 0:
    print(str(summary["Frequent Transactions in Short Time Periods"]) + " instances of rapid transactions detected."
          " This may be indicative of bot-like or automated fraud behavior.")
if summary["Unusual Locations for User's Transactions"] > 0:
    print(str(summary["Unusual Locations for User's Transactions"]) + " transactions occurred in locations inconsistent with the user's typical profile."
          " Investigate to see if these are potential account takeover events.")
if summary["Irregular Product Category Purchases"] > 0:
    print(str(summary["Irregular Product Category Purchases"]) + " purchases were outside the user's usual product categories."
          " This pattern might reflect unauthorized access or misuse of the account.")
if summary["Multiple Payment Methods for a Single User"] > 0:
    print(str(summary["Multiple Payment Methods for a Single User"]) + " users utilized multiple payment methods."
          " Behavioral inconsistency in payment methods could indicate possible account sharing or compromise.")


--- Decision Summary ---
447 transactions occurred in locations inconsistent with the user's typical profile. Investigate to see if these are potential account takeover events.
1011 purchases were outside the user's usual product categories. This pattern might reflect unauthorized access or misuse of the account.
1666 users utilized multiple payment methods. Behavioral inconsistency in payment methods could indicate possible account sharing or compromise.
